In [32]:
import duckdb
import requests

## Trabalhando os dados de notificação de dengue, para agrupar por bairro

In [33]:
# Dentre todos os casos reportados de dengue, filtramos somente por Manaus
# Identificado pelo código do IBGE (130260)

manaus_codigo = 130260
df_manaus = duckdb.sql(f"SELECT * FROM 'datalake/raw/dengue/*.csv' where ID_MUNICIP = {manaus_codigo}").df()
df_manaus.head()

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,EVIDENCIA,PLAQ_MENOR,CON_FHD,COMPLICA,TP_SISTEMA,NDUPLIC_N,DT_DIGITA,CS_FLXRET,FLXRECEBI,MIGRADO_W
0,2,A90,2023-03-24,202312,2023,13,130260,5584,5222710,2023-03-20,...,None,None,None,None,2,<NA>,2023-03-27,0,None,None
1,2,A90,2023-03-27,202313,2023,13,130260,5584,2017687,2023-03-23,...,None,None,None,None,2,<NA>,2023-03-27,1,None,None
2,2,A90,2023-01-08,202302,2023,13,130260,5584,7847866,2023-01-03,...,None,None,None,None,2,<NA>,2024-01-10,0,None,None
3,2,A90,2023-02-09,202306,2023,13,130260,5584,2013606,2023-02-03,...,None,None,None,None,2,<NA>,2023-02-09,1,None,None
4,2,A90,2023-02-22,202308,2023,13,130260,5584,2018071,2023-02-18,...,None,None,None,None,2,<NA>,2023-02-27,0,None,None


In [34]:
# Buscando o CEP de todos os CNES cadastrados 
df_estabelecimentos = duckdb.sql("SELECT DISTINCT CNES, cod_cep FROM 'datalake/raw/cnes/cnes_2024_Estabelecimentos.csv'").df()
df_estabelecimentos.head()

,CNES,COD_CEP
0,2834499,69400970
1,5016363,69400338
2,7617399,69400797
3,0115045,69057015
4,0317209,69057000


In [35]:
# Unindo os DataFrames de dengue e estabelecimentos
df = duckdb.sql("""
SELECT *
FROM df_manaus
INNER JOIN df_estabelecimentos as e ON df_manaus.id_unidade = e.CNES
""").df()

df.head()

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,CON_FHD,COMPLICA,TP_SISTEMA,NDUPLIC_N,DT_DIGITA,CS_FLXRET,FLXRECEBI,MIGRADO_W,CNES,COD_CEP
0,2,A90,2023-03-24,202312,2023,13,130260,5584,5222710,2023-03-20,...,<NA>,<NA>,2,<NA>,2023-03-27,0,<NA>,<NA>,5222710,69047481
1,2,A90,2023-03-27,202313,2023,13,130260,5584,2017687,2023-03-23,...,<NA>,<NA>,2,<NA>,2023-03-27,1,<NA>,<NA>,2017687,69080030
2,2,A90,2023-01-08,202302,2023,13,130260,5584,7847866,2023-01-03,...,<NA>,<NA>,2,<NA>,2024-01-10,0,<NA>,<NA>,7847866,69099270
3,2,A90,2023-02-09,202306,2023,13,130260,5584,2013606,2023-02-03,...,<NA>,<NA>,2,<NA>,2023-02-09,1,<NA>,<NA>,2013606,69040000
4,2,A90,2023-02-22,202308,2023,13,130260,5584,2018071,2023-02-18,...,<NA>,<NA>,2,<NA>,2023-02-27,0,<NA>,<NA>,2018071,69036110


In [36]:
# Enriquecendo os dados com a geolocalização dos CEPs
df_cep = duckdb.sql("SELECT DISTINCT COD_CEP FROM df").df()

In [37]:
def get_bairro(cep: str) -> str | None:
    url = f"https://viacep.com.br/ws/{cep}/json/"

    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()

        data = response.json()

        if data.get("erro"):
            return None

        return data.get("bairro")

    except requests.RequestException as e:
        print(f"Erro na requisição: {e}")
        return None


In [38]:
df_cep['bairro'] = df_cep['COD_CEP'].apply(get_bairro)

In [39]:
df_cep.head()

,COD_CEP,bairro
0,69040000,Dom Pedro I
1,69057001,Adrianópolis
2,69025050,Presidente Vargas
3,69087082,Tancredo Neves
4,69088754,Jorge Teixeira


In [48]:
df_pop = duckdb.sql("""
SELECT
    "População" as populacao,
    "Bairro" as bairro,
    "Zona administrativa" as zona_administrativa,
FROM read_csv_auto('datalake/raw/populacao/bairros_manaus.csv') b
""").df()
df_pop.head()

,populacao,bairro,zona_administrativa
0,10459,Adrianópolis,Centro-Sul
1,24417,Aleixo,Centro-Sul
2,76392,Alvorada,Centro-Oeste
3,33441,Armando Mendes,Leste
4,12940,Betânia,Sul


In [49]:
# Unindo os dados de dengue com os bairros dos CEPs
# DT_NOTIFIC, NU_ANO, ID_UNIDADE, CNES, df.COD_CEP, bairro
df_final = duckdb.sql('''
SELECT DT_NOTIFIC, NU_ANO, ID_UNIDADE, CNES, df.COD_CEP, df_cep.bairro, populacao, zona_administrativa
FROM df
INNER JOIN df_cep ON df.cod_cep = df_cep.COD_CEP
INNER JOIN df_pop ON df_cep.bairro = df_pop.bairro
''').df()
df_final.head()

,DT_NOTIFIC,NU_ANO,ID_UNIDADE,CNES,COD_CEP,bairro,populacao,zona_administrativa
0,2023-03-24,2023,5222710,5222710,69047481,Redenção,41572,Centro-Oeste
1,2023-03-27,2023,2017687,2017687,69080030,Coroado,60709,Leste
2,2023-01-08,2023,7847866,7847866,69099270,Novo Aleixo,114209,Norte
3,2023-02-22,2023,2018071,2018071,69036110,Compensa,89645,Oeste
4,2023-03-05,2023,2018071,2018071,69036110,Compensa,89645,Oeste


In [50]:
df_final.to_csv('datalake/processed/dengue_manaus_bairros.csv', index=False)

## trabalhando os dados de clima

In [27]:
df_clima = duckdb.sql("""
SELECT * FROM read_csv('datalake/raw/clima/INMET_N_AM_A101_MANAUS*.CSV', skip=9, all_varchar=true)
""").df()
df_clima.head()

,column00,column01,column02,column03,column04,column05,column06,column07,column08,column09,column10,column11,column12,column13,column14,column15,column16,column17,column18,column19
0,2023/01/01,0000 UTC,",2",1005,1005,"1004,7",0,24,"22,9",24,"23,8","22,9","22,4",94,91,93,55,"3,1",",8",None
1,2023/01/01,0100 UTC,0,"1005,8","1005,8",1005,0,"23,9","23,1",24,"23,7","23,1","22,8",96,93,95,336,"1,4",",4",None
2,2023/01/01,0200 UTC,0,"1005,8","1006,1","1005,8",0,"23,9","22,9","24,2","23,9","23,3","22,9",96,94,95,12,"1,5",",4",None
3,2023/01/01,0300 UTC,0,"1005,6","1005,8","1005,6",0,"24,3","22,9","24,4","23,8","23,2","22,9",95,92,92,64,"2,6","1,5",None
4,2023/01/01,0400 UTC,",4","1005,8","1005,9","1005,6",0,"23,8","22,8","24,3","23,8","23,1","22,7",94,92,94,50,3,"1,5",None


In [28]:
columns = [
    'Data',
    'Hora UTC',
    'PRECIPITACAO_TOTAL_HORARIO_mm',
    'PRESSAO_ATMOSFERICA_NIVEL_ESTACAO_mB',
    'PRESSAO_MAX_NA_HORA_ANT_mB',
    'PRESSAO_MIN_NA_HORA_ANT_mB',
    'RADIACAO_GLOBAL_Kj_m2',
    'TEMPERATURA_AR_BULBO_SECO_C',
    'TEMPERATURA_PONTO_ORVALHO_C',
    'TEMPERATURA_MAX_HORA_ANT_C',
    'TEMPERATURA_MIN_HORA_ANT_C',
    'TEMPERATURA_ORVALHO_MAX_HORA_ANT_C',
    'TEMPERATURA_ORVALHO_MIN_HORA_ANT_C',
    'UMIDADE_REL_MAX_HORA_ANT_percent',
    'UMIDADE_REL_MIN_HORA_ANT_percent',
    'UMIDADE_RELATIVA_AR_percent',
    'VENTO_DIRECAO_HORA_gr',
    'VENTO_RAJADA_MAXIMA_m_s',
    'VENTO_VELOCIDADE_HORA_m_s',
    '-'
]

df_clima.columns = columns
df_clima['AnoMesProcessamento'] = pd.to_datetime(df_clima['Data']).dt.strftime('%Y-%m')

df_clima = duckdb.sql("""
SELECT
  AnoMesProcessamento as mes,
  AVG(replace(TEMPERATURA_AR_BULBO_SECO_C, ',', '.')::DOUBLE) AS TEMPERATURA_AR_BULBO_SECO_C,
  AVG(replace(UMIDADE_RELATIVA_AR_percent, ',', '.')::DOUBLE) AS UMIDADE_RELATIVA_AR_percent,
  AVG(replace(PRECIPITACAO_TOTAL_HORARIO_mm, ',', '.')::DOUBLE) AS PRECIPITACAO_TOTAL_HORARIO_mm,
  AVG(replace(RADIACAO_GLOBAL_Kj_m2, ',', '.')::DOUBLE) AS RADIACAO_GLOBAL_Kj_m2,
  AVG(replace(VENTO_RAJADA_MAXIMA_m_s, ',', '.')::DOUBLE) AS VENTO_RAJADA_MAXIMA_m_s

FROM
  df_clima
GROUP BY AnoMesProcessamento
""").df()
df_clima = df_clima.sort_values('mes')
df_clima.head()

,mes,TEMPERATURA_AR_BULBO_SECO_C,UMIDADE_RELATIVA_AR_percent,PRECIPITACAO_TOTAL_HORARIO_mm,RADIACAO_GLOBAL_Kj_m2,VENTO_RAJADA_MAXIMA_m_s
19,2023-01,25.890897,83.884138,0.374069,515.180000,3.680000
12,2023-02,25.970399,85.187117,0.542331,539.477914,3.632362
20,2023-03,26.533697,83.757162,0.560982,533.327694,3.777217
13,2023-04,27.054455,81.850071,0.387270,560.903536,3.689235
4,2023-05,27.847139,80.115804,0.183924,511.275034,3.768076


In [29]:
df_clima.to_csv('datalake/processed/clima_manaus.csv', index=False)